# Modeling

## Reference

The code in this notebook is adapted and modified from the following Youtube tutorial: https://www.youtube.com/watch?v=doDUihpj6ro 

## Usage

In this notebook, an LSTM based neural network model can be trained with data generated by the notebook "1_Example-Data-Generation.ipynb"

The path to the input data `DATA_PATH` is set to 'MP_Data_test'. Change it, if you want to use different / your own data generated with "1_Example-Data-Generation.ipynb". 

The weights of the trained model are saved with the name `model_name = 'first_model_whoop_whoop.h5'`. Change it, if you want ;)

## 1. Install and Import Dependencies

### Install Dependencies

In [9]:
%pip install tensorflow-macos opencv-python mediapipe-silicon sklearn matplotlib
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib # original code line from tutorial (he used a Windows system)

Note: you may need to restart the kernel to use updated packages.


### Import Dependencies

In [10]:
# general 
import numpy as np
import pandas as pd
import os # easier file path handling

# for device camera feed
import cv2 # opencv
from matplotlib import pyplot as plt # imshow for easy visualization
import time # to insert breaks / "sleep" in between frames
import mediapipe as mp # for accessing and reading from device camera

# for data pre-processing
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# for model building and training
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# for model evaluation
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay, accuracy_score

## 2. Preprocess Data and Create Labels and Features

### Some global setup (edit to your needs)

In [26]:
# path for input data
#DATA_PATH = os.path.join('MP_Data')
DATA_PATH = os.path.join('../data/')


# actions to detect (get later from our .json file)
#actions = np.array(['hello', 'thanks', 'iloveyou'])
actions = np.array ( ['alligator', 'radio', 'moon', 'sleep', 'grandpa', 'tiger', 'pencil', 'sleepy', 'grandma', 'chocolate'])
n_sequences = 30 # 30 videos per sequence / word / sign (get it later from input file)
sequence_length = 30 # each video with 30 frames (get it later from input file)
n_keypoints = 1086 # kaggle: 1086; tutorial: 1662

label_map = {label:num for num, label in enumerate(actions)} # create label map (dict, later our .json file)


### Load X and y Data

In [27]:
#loading our preprocessed kaggle data
X = np.load(os.path.join(DATA_PATH, 'X-data.npy'))
y = np.load(os.path.join(DATA_PATH, 'y-data.npy'))

### Splitting Train and Test Data

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## Build and Train LSTM Neural Network

### Setup Callbacks

In [14]:
# for tensorflow callbacks
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

### Model building

In [15]:
# setup sequential model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(n_sequences, n_keypoints))) # input_shape=(#sequences, #keypoints)
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # actions.shape[0]: from user-defined actions (see above)

# compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# categorical_crossentropy must be used for multiclass classification model! 

# show summary of model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            294656    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 449,450
Trainable params: 449,450
Non-trai

### Model fitting

In [16]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
# advantage of using mediapipe holistic model is you don't need additional data generator to build up a pipeline of data. Training data fits all into memory.
# 2023-04-06-23-43 run-time: 

Epoch 1/2000


2023-04-06 23:42:35.880135: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 3s 38ms/step - loss: 13.3999 - categorical_accuracy: 0.0952
Epoch 2/2000
10/10 [==============================] - 0s 43ms/step - loss: 10.5158 - categorical_accuracy: 0.1048
Epoch 3/2000
10/10 [==============================] - 0s 40ms/step - loss: 52.1858 - categorical_accuracy: 0.0762
Epoch 4/2000
10/10 [==============================] - 0s 38ms/step - loss: 73.9521 - categorical_accuracy: 0.1143
Epoch 5/2000
10/10 [==============================] - 0s 44ms/step - loss: 97.6091 - categorical_accuracy: 0.1270
Epoch 6/2000
10/10 [==============================] - 0s 39ms/step - loss: 108.5172 - categorical_accuracy: 0.1238
Epoch 7/2000
10/10 [==============================] - 0s 47ms/step - loss: 86.5638 - categorical_accuracy: 0.1206
Epoch 8/2000
10/10 [==============================] - 0s 43ms/step - loss: 66.7620 - categorical_accuracy: 0.0730
Epoch 9/2000
10/10 [==============================] - 0s 39ms/step - loss: 47.7697 - categorical_acc

### Some Tensorboard stuff I didn't figure out (not important)

In [17]:
# %load_ext tensorboard

In [18]:
# to run it in the notebook: 
# %tensorboard --logdir=./Logs/train --port=8008

# to run it in terminal: python3 -m tensorboard --logdir=./Logs/train --port=8008
# then copy+paste this into your internet browser: localhost:8008

## Prediction

In [19]:
y_pred = model.predict(X_test)

1/1 [==============================] - 0s 300ms/step


### Looking at a single example prediction

In [20]:
actions[np.argmax(y_pred[0])] # prediction

'sleepy'

In [21]:
actions[np.argmax(y_test[0])] # actual

'pencil'

## Evaluation using Confusion Matrix and Accuracy

### back-converting one-hot codes to labels

In [22]:
# convert one-hot encoded categories back to labels, 
# e.g. 0, 1 and 2 instead of [1,0,0], [0,1,0], [0,0,1]
y_test_original = y_test.copy()
y_pred_original = y_pred.copy()
y_test = np.argmax(y_test, axis=1).tolist()
y_pred = np.argmax(y_pred, axis=1).tolist()

### Multilabel Confusion Matrix

In [23]:
# multilabel confusion matrix 
multilabel_confusion_matrix(y_test, y_pred)

array([[[14,  0],
        [ 3,  0]],

       [[16,  0],
        [ 1,  0]],

       [[13,  0],
        [ 4,  0]],

       [[16,  0],
        [ 1,  0]],

       [[16,  0],
        [ 1,  0]],

       [[13,  0],
        [ 4,  0]],

       [[ 0, 15],
        [ 0,  2]],

       [[16,  0],
        [ 1,  0]]])

### Accuracy Score

In [24]:
# accuracy score
accuracy_score(y_test, y_pred)

0.11764705882352941

## Save Model Weights

In [25]:
# save model
model_name = 'Easter_model_10signs_j.h5'
model.save(model_name)